In [1]:
import json
import os
import requests
import numpy as np
import pandas as pd
import bz2


In [2]:
# Generate a list of URLs for the reddit files

url_stub = 'https://files.pushshift.io/reddit/comments/'
file_urls = []

for year in range(2006,2020):
    
    # Set extension by year for most files
    
    if year < 2018:
        extension = '.bz2'
    elif year < 2019:
        extension = '.xz'
    else:
        extension = '.zst'
    
    # Loop over each file and print its name
    for month in range(1,13):
        
        # Handle a few special cases    
        if (year == 2017 and month == 12):
            extension = '.xz'
        if (year == 2018 and month in [11,12]):
            extension = '.zst'

        # Create the file name, adding the leading zero
        # if the month is 1 - 9
        if month < 10:
            file = 'RC_' + str(year) + '-0' + str(month) + extension
        else:
            file = 'RC_' + str(year) + '-' + str(month) + extension
            
        file_urls.append(url_stub + file)

In [3]:
%%script false --no-raise-error

# Files have already been retrieved and saved

# Retrieve a sample file from each year
# The month is staggered so a different month
# will be chosen for each year

for i in range(0,169, 11):
    
    response = requests.get(file_urls[i])
    
    # Extract the file name from the URL and generate
    # the save path
    file_download = file_urls[i].split('/')[5]
    file_saved = 'Reddit_Posts\\zipped\\' + file_download
    
    # Write the file (i.e. bytes)
    # This is very slow for large files and may hang
    with open(file_saved, 'wb') as f:
        f.write(response.content)

In [14]:
# Decompress one file as a test
# content will be a byte stream

file_compressed = 'Reddit_Posts\\zipped\\RC_2006-01.bz2'

with bz2.open(file_compressed,'rb') as f:
    content = f.read()



In [9]:
# Extract the first record from the content

record = ''

for i in range(0, 1000):
    
    # Needed so char is a character, not an integer
    char = content.decode()[i]
    
    # Concatenate each character, breaking on the '}' (i.e. end of record)
    if char != '}':
        record += char
    elif char == '}':
        record += char
        break


In [ ]:
print(content.decode())

str